In [1]:
import os
import dotenv
dotenv.load_dotenv()

True

In [2]:
files = []
file_path = os.path.join(os.getcwd(), "doctrine")
file_list = os.listdir(file_path)

for name in file_list :
    files.append(os.path.join(file_path, name))

files

['c:\\Users\\dudwn\\Desktop\\github\\Project\\doctrine\\MCRP 3-10A.1.pdf']

In [3]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
for file in files : 
    loader = PyPDFLoader(file)
    document = loader.load()

In [5]:
document[15]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2020-08-25T07:42:41-04:00', 'moddate': '2020-08-26T09:17:40-04:00', 'source': 'c:\\Users\\dudwn\\Desktop\\github\\Project\\doctrine\\MCRP 3-10A.1.pdf', 'total_pages': 312, 'page': 15, 'page_label': '16'}, page_content='1-5 \n \nThe range of the battalion’s employment is primarily limited by its size, combat power, manner \nof configuration, resources, assets, and personnel.  In its basic foot-mobile configuration, it is \nlimited by its: \n \n\uf0b7 Logistic assets. \n\uf0b7 Command and control (C2) assets. \n\uf0b7 Mobility assets. \n\uf0b7 Vulnerability to armor, artillery, and air threats. \n\uf0b7 Vulnerability to chemical, biological, radiological, and nuclear (CBRN) attack and \npossession of only limited decontamination capabilities. \n \nAugmentation \nAn infantry battalion often serves as the nucleus of a task force or landing team by attaching \nadditional capabilities.  The only limit to the amount 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 300)
texts = text_splitter.split_documents(document)
#페이지 단위로 문자가 찢어지는 문제 해결 필요.

In [7]:
print(texts[798])

page_content='support the movement of another element with immediate fire.  Every battalion evolution or 
activity should employ an overwatch element, whether it is annual training on a live-fire range, a 
change of command ceremony, or the battalion commander conducting a KLE with village 
elders.  The need for overwatch cannot be dismissed.  The size of an overwatch element can 
range from two Marines to the employment of a platoon or more, based on METT-T' metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2020-08-25T07:42:41-04:00', 'moddate': '2020-08-26T09:17:40-04:00', 'source': 'c:\\Users\\dudwn\\Desktop\\github\\Project\\doctrine\\MCRP 3-10A.1.pdf', 'total_pages': 312, 'page': 237, 'page_label': '238'}


In [8]:
db_dir = os.path.join(os.getcwd(), "db")

In [9]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [10]:
def create_vector_store(docs, embedding, store_name, initialize = False) :
    persistent_directory = os.path.join(db_dir, store_name)
    if initialize == True :
        os.remove(persistent_directory)
        print(f"-----Vector Store {store_name} is Initialized-----")

    if not os.path.exists(persistent_directory) :
        print(f"-----Creating Vector Store {store_name}-----")
        Chroma.from_documents(
            docs, embedding, persist_directory = persistent_directory
        )
        print(f"-----Finished Creating Vector Store {store_name}-----")
    else :
        print(
            f"Vector Store {store_name} already exists. No need to Initialize"
        )

In [11]:
print("-----Using OpenAI Embeddings")
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
create_vector_store(texts, openai_embeddings, "chroma_db_openai")

-----Using OpenAI Embeddings
Vector Store chroma_db_openai already exists. No need to Initialize


In [13]:
persistent_directory = os.path.join(db_dir, "chroma_db_openai")


def query_vector_store(store_name, query, embeddings, search_type, search_kwargs) :
    if os.path.exists(persistent_directory) : 
        print(f"----- Querying the Vectore Store {store_name} -----")
        db = Chroma(
            persist_directory = persistent_directory,
            embedding_function = embeddings
        )
        retriever = db.as_retriever(
            search_type = search_type,
            search_kwargs = search_kwargs
        )
        relevant_docs = retriever.invoke(query)
        print(f"----- Relevant Documents for {store_name} -----")
        for i, doc in enumerate(relevant_docs, 1) : 
            print(f"Document {i} : {doc.page_content}")
            if doc.metadata : 
                print(f"Source : {doc.metadata.get("source", "Unknown")}\n")
        return relevant_docs
    else : 
        print(f"Vector store {store_name} does not exist.")

In [14]:
query = "what is overwatch?"

In [15]:
relevant_docs = query_vector_store("chroma_db_metadata", query, openai_embeddings, "similarity", {"k" : 5})

----- Querying the Vectore Store chroma_db_metadata -----


C:\Users\dudwn\AppData\Local\Temp\ipykernel_33276\2670307415.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


----- Relevant Documents for chroma_db_metadata -----
Document 1 : 11-14 
 
considerations.  They may be armed or unarmed, depending on the situation.  The following 
guidelines should apply to overwatch employment: 
 
 Personnel and equipment remain tactically alert to provide security, protection, and  
early warning.  
 Overwatch elements should be given guidance and training regarding the use of deadly 
force, local ROE, and escalation of force continuum procedures. 
 Overwatch elements rehearse the immediate actions required of them. 
 Units must employ the appropriate communications to ensure that overwatch elements 
can provide proper early warning of a threat. 
 Overwatch elements remain vigilant and alert to potentially hostile activities and actions 
near their units. 
 Overwatch elements should be rotated to maintain the highest levels of awareness. 
 Overwatch elements should be employed at all unit evolutions, regardless of size or 
operational environment.  
 
Frat

In [16]:
combined_input = (
    "Here are some documents that might help answer the question : "
    + query
    + "\n\n Relevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\n Please provide an answer based only on the provided documents"
)

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage


In [19]:
messages = [
    SystemMessage(content = "You are a helpful assistant"),
    HumanMessage(content = combined_input)
]

In [ ]:
from langchain_openai import ChatOpenAI

In [23]:
model = ChatOpenAI()

In [28]:
messages = model.invoke(messages)

In [36]:
messages.content

'Based on the provided documents, Overwatch is defined as a tactical movement technique where one element is positioned to support the movement of another element with immediate fire. It is a crucial aspect of military operations, requiring personnel and equipment to remain tactically alert to provide security, protection, and early warning. Overwatch elements must be well-trained in the use of deadly force, local rules of engagement (ROE), and escalation of force continuum procedures. Units should employ appropriate communications to ensure overwatch elements can provide early warning of threats, and rotation of overwatch elements is necessary to maintain high levels of awareness. Overwatch should be utilized in all unit evolutions, regardless of size or operational environment. The size of an overwatch element can vary based on mission requirements.'

In [ ]:
chcp 65001

bundle exec jekyll serve